In [2]:
! pip install qutip qutip_qip matplotlib qiskit qutip-qtrl tqdm qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.3 MB/s eta 0:00:00


In [3]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qutip_qip.qiskit import QiskitCircuitSimulator,QiskitPulseSimulator

import numpy as np
from typing import List,Optional,Union ,Dict, Any
import matplotlib.pyplot as plt
import qutip
from qutip import basis, fidelity,sigmax, sigmay, sigmaz,identity,rand_unitary,tensor,mesolve,Qobj, Bloch
from qutip_qip.operations import  rx,ry,rz,cz_gate,expand_operator
from typing import List
from tqdm import tqdm
from qutip_qip.device import Processor, Model
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
import qiskit

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Exercise 1

In [5]:
from scipy.optimize import minimize
import numpy as np
import qutip
from qutip import basis, mesolve, fidelity

# Define the barebone simulation function for the rx gate
def simulate_rx(Omega_01, theta, initial_state):
    GROUND = basis(2, 0)
    EXCITED = basis(2, 1)
    collapse_ops = []
    H = Omega_01 / 2 * (GROUND * EXCITED.dag() + EXCITED * GROUND.dag())
    result = mesolve(
        H=H,
        rho0=initial_state,
        tlist=np.array([0, theta / 10]),
        c_ops=collapse_ops,
        options={'store_final_state': True}
    )
    return result.final_state

# Define the target states for optimization
thetas = [np.pi / 2, np.pi / 4, np.pi / 6]
target_states = [rx(theta) * basis(2, 0) for theta in thetas]

# Define the fidelity function for optimization
def fidelity_function(x):
    Omega_01= x[0]
    initial_state = basis(2, 0)
    fidelities = []
    for theta, target_state in zip(thetas, target_states):
        final_state = simulate_rx(Omega_01, theta, initial_state)
        fidelities.append(fidelity(final_state, target_state))
    return -np.mean(fidelities)


# Optimize Omega_01
initial_Omega_01 = 1
result = minimize(fidelity_function, [initial_Omega_01])
print(f"Optimized Omega_01: {result.x[0]}, fun: {result.fun}")

Optimized Omega_01: 9.999383377170595, fun: -0.9999999994694106


In [6]:
# Define the barebone simulation function for the rz gate
def simulate_rz(delta_1, theta, initial_state):
    EXCITED = basis(2, 1)
    collapse_ops = []
    H = delta_1 * EXCITED * EXCITED.dag()
    result = mesolve(
        H=H,
        rho0=initial_state,
        tlist=np.array([0, theta / 10]),
        c_ops=collapse_ops,
        options={'store_final_state': True}
    )
    return result.final_state

# Define the target states for optimization
thetas = [np.pi / 2, np.pi / 4, np.pi / 6]
target_states = [rz(theta) * (basis(2, 0)+basis(2, 1)).unit() for theta in thetas]

# Define the fidelity function for optimization
def fidelity_function(x):
    delta_1 = x[0]
    initial_state = (basis(2, 0)+basis(2, 1)).unit()
    fidelities = []
    for theta, target_state in zip(thetas, target_states):
        final_state = simulate_rz(delta_1, theta, initial_state)
        fidelities.append(fidelity(final_state, target_state))
    return -np.mean(fidelities)

# Optimize delta_1
initial_delta_1 = 1
result = minimize(fidelity_function, [initial_delta_1])
print(f"Optimized delta_1: {result.x[0]}, fun: {result.fun}")

Optimized delta_1: -9.997619824080692, fun: -0.9999999921066998


In [7]:
import numpy as np
import uuid
import random
from collections import Counter

import qutip
import qiskit
from qutip import basis
from qutip_qip.circuit import QubitCircuit
from qutip_qip.circuit.circuitsimulator import CircuitResult
from qutip_qip.device import Processor

from qutip_qip.qiskit import QiskitSimulatorBase
from qutip_qip.qiskit.job import Job
from qutip_qip.qiskit.converter import convert_qiskit_circuit
from qiskit.providers import BackendV1, Options
from qiskit.providers.models import QasmBackendConfiguration
from qiskit.result import Result, Counts
from qiskit.result.models import ExperimentResult, ExperimentResultData
from qiskit.quantum_info import Statevector, DensityMatrix
from qiskit.circuit import QuantumCircuit
from qiskit.qobj import QobjExperimentHeader

class MyQiskitSimulatorBase(BackendV1):
    """
    The base class for ``qutip_qip`` based ``qiskit`` backends.
    """

    def __init__(self, configuration=None, **fields):
        if configuration is None:
            configuration_dict = self._DEFAULT_CONFIGURATION
        else:
            configuration_dict = self._DEFAULT_CONFIGURATION.copy()
            for k, v in configuration.items():
                configuration_dict[k] = v

        configuration = QasmBackendConfiguration.from_dict(configuration_dict)

        super().__init__(configuration=configuration)

        self.options.set_validator(
            "shots", (1, self.configuration().max_shots)
        )

    def run(self, qiskit_circuit: QuantumCircuit, **run_options) -> Job:
        """
        Simulates a circuit on the required backend.

        Parameters
        ----------
        qiskit_circuit : :class:`qiskit.circuit.QuantumCircuit`
            The ``qiskit`` circuit to be simulated.

        **run_options:
            Additional run options for the backend.

            Valid options are:

            shots : int
                Number of times to sample the results.
            allow_custom_gate: bool
                Allow conversion of circuit using unitary matrices
                for custom gates.

        Returns
        -------
        :class:`.Job`
            Job object that stores results and execution data.
        """
        # configure the options
        self.set_options(
            shots=(
                run_options["shots"]
                if "shots" in run_options
                else self._default_options().shots
            ),
            allow_custom_gate=(
                run_options["allow_custom_gate"]
                if "allow_custom_gate" in run_options
                else self._default_options().allow_custom_gate
            ),
        )
        
        # qutip_circ = convert_qiskit_circuit(
        #     qiskit_circuit,
        #     allow_custom_gate=self.options.allow_custom_gate,
        # )

        job_id = str(uuid.uuid4())

        # job = Job(
        #     backend=self,
        #     job_id=job_id,
        #     result=self._run_job(job_id, qutip_circ),
        # )
        job = Job(
            backend=self,
            job_id=job_id,
            result=self._run_job(job_id, qiskit_circuit),
        )
        return job

    def _sample_shots(self, count_probs: dict) -> Counts:
        """
        Sample measurements from a given probability distribution.

        Parameters
        ----------
        count_probs: dict
            Probability distribution corresponding
            to different classical outputs.

        Returns
        -------
        :class:`qiskit.result.Counts`
            Returns the ``Counts`` object sampled according to
            the given probabilities and configured shots.
        """
        shots = self.options.shots
        samples = random.choices(
            list(count_probs.keys()), list(count_probs.values()), k=shots
        )
        return Counts(Counter(samples))

    def _get_probabilities(self, state):
        """
        Given a state, return an array of corresponding probabilities.
        """
        if state.type == "oper":
            # diagonal elements of a density matrix are
            # the probabilities
            return state.diag()

        # squares of coefficients are the probabilities
        # for a ket vector
        return np.array([np.abs(coef) ** 2 for coef in state])

class MyQiskitPulseSimulator(MyQiskitSimulatorBase):
    """
    ``qiskit`` backend dealing with pulse-level simulation.

    Parameters
    ----------
    processor : :class:`.Processor`
        The processor model to be used for simulation.
        An instance of the required :class:`.Processor`
        object is to be provided after initialising
        it with the required parameters.

    configuration : dict
        Configurable attributes of the backend.

    Attributes
    ----------
    processor : :class:`.Processor`
        The processor model to be used for simulation.
    """

    processor = None
    MAX_QUBITS_MEMORY = 10
    BACKEND_NAME = "pulse_simulator"
    _DEFAULT_CONFIGURATION = {
        "backend_name": BACKEND_NAME,
        "backend_version": "0.1",
        "n_qubits": MAX_QUBITS_MEMORY,
        "url": "https://github.com/qutip/qutip-qip",
        "simulator": True,
        "local": True,
        "conditional": False,
        "open_pulse": False,
        "memory": False,
        "max_shots": int(1e6),
        "coupling_map": None,
        "description": "A qutip-qip based pulse-level \
            simulator based on the open system solver.",
        "basis_gates": [],
        "gates": [],
    }

    def __init__(self, processor: Processor, configuration=None, **fields):
        self.processor = processor
        super().__init__(configuration=configuration, **fields)

    def _parse_results(
        self, final_state: qutip.Qobj, job_id: str, qutip_circuit: QubitCircuit
    ) -> qiskit.result.Result:
        """
        Returns a parsed object of type :class:`qiskit.result.Result`
        for the pulse simulators.

        Parameters
        ----------
        density_matrix : :class:`.Qobj`
            The resulting density matrix obtained from `run_state` on
            a circuit using the Pulse simulator processors.

        job_id : str
            Unique ID identifying a job.

        qutip_circuit : :class:`.QubitCircuit`
            The circuit being simulated.

        Returns
        -------
        :class:`qiskit.result.Result`
            Result of the pulse simulation.
        """
        count_probs = {}
        counts = None

        # calculate probabilities of required states
        if final_state:
            for i, prob in enumerate(self._get_probabilities(final_state)):
                if not np.isclose(prob, 0):
                    count_probs[hex(i)] = prob
            # sample the shots from obtained probabilities
            counts = self._sample_shots(count_probs)

        exp_res_data = ExperimentResultData(
            counts=counts,
            statevector=(
                Statevector(data=final_state.full())
                if final_state.type == "ket"
                else DensityMatrix(data=final_state.full())
            ),
        )

        header = QobjExperimentHeader.from_dict(
            {
                "name": (
                    qutip_circuit.name
                    if hasattr(qutip_circuit, "name")
                    else ""
                ),
                "n_qubits": qutip_circuit.N,
            }
        )

        exp_res = ExperimentResult(
            shots=self.options.shots,
            success=True,
            data=exp_res_data,
            header=header,
        )

        result = Result(
            backend_name=self.configuration().backend_name,
            backend_version=self.configuration().backend_version,
            qobj_id=id(qutip_circuit),
            job_id=job_id,
            success=True,
            results=[exp_res],
        )

        return result

    def _run_job(self, job_id: str, 
                #  qutip_circuit: QubitCircuit
                qiskit_circuit: QuantumCircuit
                 ) -> Result:
        """
        Run a :class:`.QubitCircuit` on the Pulse Simulator.

        Parameters
        ----------
        job_id : str
            Unique ID identifying a job.

        qutip_circuit : :class:`.QubitCircuit`
            The circuit obtained after conversion
            from :class:`.QuantumCircuit` to :class:`.QubitCircuit`.

        Returns
        -------
        :class:`qiskit.result.Result`
            Result of the simulation.
        """
        zero_state = self.processor.generate_init_processor_state()

        # self.processor.load_circuit(qutip_circuit)
        self.processor.load_circuit(qiskit_circuit)
        result = self.processor.run_state(zero_state)

        final_state = self.processor.get_final_circuit_state(result.states[-1])

        qutip_circ = convert_qiskit_circuit(
            qiskit_circuit,
            allow_custom_gate=self.options.allow_custom_gate,
        )

        return self._parse_results(
            final_state=final_state, job_id=job_id, qutip_circuit=qutip_circ
        )

    @classmethod
    def _default_options(cls):
        """
        Default options for the backend.

        Options
        -------
        shots : int
            Number of times to sample the results.

        allow_custom_gate : bool
            Allow conversion of circuit using unitary matrices
            for custom gates.
        """
        return Options(shots=1024, allow_custom_gate=True)

GROUND = basis(4, 0)
EXCITED = basis(4, 1)
LEAKAGE = basis(4, 2)
RYDBERG = basis(4, 3)


class minimal_processor(Processor):
    def __init__(self,
                 num_qubits=2,
                 Omega_01 = 9.999385781382449,
                 delta_1 = -9.997619824080692,
                 gamma_r = 1 / 540):
        self.Omega_01 = Omega_01
        self.delta_1= delta_1
        self.basis_gates=['rx','rz','cz']
        self.qbt_dim=4
        self.model=Model(num_qubits=num_qubits,
                         dims=[self.qbt_dim for _ in range(num_qubits)])
        self.num_qubits=num_qubits
        self.get_c_ops(gamma_r)

    def evolve(self,
               state:qutip.Qobj,
                ins: qiskit._accelerate.circuit.CircuitInstruction):
        name = ins.operation.name
        param = next(iter(ins.operation.params), None)
        qbts = [qubit._index for qubit in ins.qubits]

        # print(f"{name}, {param}, {qbts}")

        if name == 'rx':
            Omega_01 =  self.Omega_01
            return mesolve(
                    H=self.id_wrap(Omega_01/2 * (GROUND * EXCITED.dag() + EXCITED * GROUND.dag()),
                                   qbts[0]), 
                    rho0=state, 
                    tlist=np.array([0, param/10]), 
                    c_ops=self.collapse_ops, 
                    options={'store_final_state':True}
                )
        elif name == 'rz':
            delta_1 = self.delta_1
            H=self.id_wrap(delta_1 * EXCITED * EXCITED.dag(),
                                   qbts[0])
            # print(H)
            return mesolve(
                    H=H, 
                    rho0=state, 
                    tlist=np.array([0,param/10]), 
                    c_ops=self.collapse_ops, 
                    options={'store_final_state':True}
                )
        elif name == 'cz':
            t_tot = 0.540 #microsecond, which is the total duration of the gate protocol
            B=200*2*np.pi #interaction strength
            omegaMax = 17*2*np.pi #Mhz
            deltaMax = 23*2*np.pi #Mhz
            tau = 0.175*t_tot 
            a = np.exp(-(t_tot/4)**4/tau**4)
            def Rabi_frequency(t,args):
                return (t<t_tot/2)*omegaMax*(np.exp(-(t-t_tot/4)**4/tau**4)-a)/(1-a)    \
                +    (t>=t_tot/2)*omegaMax*(np.exp(-(t-3*t_tot/4)**4/tau**4)-a)/(1-a)
            def Detunning(t,args):
                return (t<t_tot/2)*(-1)*deltaMax*np.cos(2*np.pi*t/t_tot) \
                        +    (t>=t_tot/2)*deltaMax*np.cos(2*np.pi*t/t_tot)
            H = []
            for idx in qbts:
                H.append([self.id_wrap(0.5*RYDBERG*EXCITED.dag()+0.5*EXCITED*RYDBERG.dag(), 
                                       idx),
                          Rabi_frequency])
                H.append([self.id_wrap(RYDBERG*RYDBERG.dag(), 
                                       idx),
                          Detunning])
                
            H.append(
                self.id_wrap(B*tensor(RYDBERG,RYDBERG)*tensor(RYDBERG,RYDBERG).dag(),qbts)
            ) # This assumes the system involves two qubit only.
            # print(H)
            # print(self.collapse_ops)
            return mesolve(
                    H=H, 
                    rho0=state, 
                    tlist=np.array([0, t_tot]), 
                    c_ops=self.collapse_ops, 
                    options={'store_final_state':True,'nsteps':10000}  
                )
            
    
    def run_state(self,
                  init_state:qutip.Qobj) -> qutip.solver.Result:
        if init_state.isket:
            assert init_state.dims == [[self.qbt_dim for _ in range(self.num_qubits)],[1 for _ in range(self.num_qubits)]]
        else:
            assert init_state.dims == [[self.qbt_dim for _ in range(self.num_qubits)],[self.qbt_dim for _ in range(self.num_qubits)]]
        state = init_state
        for ins in tqdm(self.qiskit_circ_transpiled.data,'Simulating gate'):
            result = self.evolve(state,ins)
            state = result.final_state
        return result
    
    def generate_init_processor_state(self)->qutip.Qobj:
        # Always initialize in zero
        return tensor([basis(self.qbt_dim, 0) for _ in range(self.num_qubits)])
    
    def id_wrap(self,
                op:qutip.Qobj,
                idx:Union[int,List[int]])->qutip.Qobj:
        if isinstance(idx,int):
            idx = [idx]
        return expand_operator(oper=op,
                               dims=[self.qbt_dim for _ in range(self.num_qubits)],
                               targets=idx)

    
    def get_c_ops(self,gamma_r):
        L0 = np.sqrt(1/16 * gamma_r) * (GROUND * RYDBERG.dag())
        L1 = np.sqrt(1/16 * gamma_r) * (EXCITED * RYDBERG.dag())
        Ld = np.sqrt(7/8 * gamma_r) * (LEAKAGE * RYDBERG.dag())
        single_q_c_ops = [L0, L1, Ld]

        self.collapse_ops = []
        for q in range(self.num_qubits):
            for c_op in single_q_c_ops:
                self.collapse_ops.append(self.id_wrap(c_op,
                                                      q))
    
    def load_circuit(self,
                     qiskit_circuit:qiskit.QuantumCircuit)->None:
        self.qiskit_circ_transpiled = transpile(qiskit_circuit,basis_gates=self.basis_gates)
        # self.qiskit_circ_transpiled = qiskit_circ_transpiled
    
    def get_final_circuit_state(self,state:qutip.Qobj)->qutip.Qobj:
        rho = state
        if rho.isket:
            rho = qutip.ket2dm(rho)
        rho_arr = rho.full()
        rho_reshaped = rho_arr.reshape(*[self.qbt_dim for _ in range(self.num_qubits*2)])
        # print(rho_reshaped.shape)
        rho_reshaped_2lvl = rho_reshaped[tuple(slice(0, 2) for _ in range(2 * self.num_qubits))]
        # print(rho_reshaped_2lvl.shape)
        return Qobj(rho_reshaped_2lvl.reshape(2**self.num_qubits , 2**self.num_qubits), dims=[[2] * self.num_qubits, [2] * self.num_qubits])
    

In [8]:
from qiskit_aer import Aer
class BasicSimulator:
    def run(self, circuit: QuantumCircuit):
        # Simulate the circuit
        simulator = Aer.get_backend('statevector_simulator')
        result = simulator.run(circuit,shots=1024).result()
        
        # Extract statevector and counts
        statevector = result.get_statevector(circuit)        
        shots = 1024
        count_probs = result.get_counts(circuit)
        samples = random.choices(
            list(count_probs.keys()), list(count_probs.values()), k=shots
        )
        counts = Counts(Counter(samples))

        # Store results in a dictionary
        return {
            'statevector': statevector,
            'counts': counts
        }

In [16]:
circ1 = QuantumCircuit(2)
circ1.h(0)
circ1.h(1)

circ2 = QuantumCircuit(2)
# circ2.h(0)
circ2.rx(0.01, 1)
circ2.h(1)
circ2.rz(0.01, 1)

circ3 = QuantumCircuit(3)
circ3.h(0)
circ3.h(1)
circ3.rz(0.5,1)
circ3.rz(1.2,1)

In [17]:
import itertools
for circ in [circ1,circ2,circ3]:
    processor = minimal_processor(circ.num_qubits,gamma_r=0)
    pulse_backend = MyQiskitPulseSimulator(processor)
    job_pulse = pulse_backend.run(circ)
    result_pulse = job_pulse.result()
    pulse_state = result_pulse.data()['statevector'].data
    pulse_state_count = result_pulse.data()['counts']
    
    # Use the BasicSimulator to run the circuit
    basic_simulator = BasicSimulator()
    result_basic = basic_simulator.run(circ)
    circ_state = result_basic['statevector']
    circ_state_count = result_basic['counts']

    print(f'Pulse state fidelity: {fidelity(qutip.Qobj(pulse_state),qutip.Qobj(circ_state))}')

    def get_count(counts,s):
        padded_string = s.zfill(circ.num_qubits)
        return counts.get(padded_string,0)
    
    binary_strings = [''.join(bits) for bits in itertools.product('01', repeat=circ.num_qubits)]

    print(f'Root_mean_square distance: {np.sqrt(np.sum([(get_count(pulse_state_count,s)/1024-get_count(circ_state_count,s)/1024)**2 for s in binary_strings])/circ.num_qubits**2)}')

Simulating gate: 100%|██████████| 6/6 [00:00<00:00, 445.26it/s]


Pulse state fidelity: 1.000000425618799
Root_mean_square distance: 0.17614234287707098


Simulating gate: 100%|██████████| 3/3 [00:00<00:00, 478.64it/s]


Pulse state fidelity: 0.5000231599179342
Root_mean_square distance: 0.3544753730085915


Simulating gate: 100%|██████████| 6/6 [00:00<00:00, 132.68it/s]

Pulse state fidelity: 0.5000219832053322
Root_mean_square distance: 0.20466512439656148
